In [1]:
# multi_video_player_tool.ipynb가 들어있는 폴더의 절대 경로
present_folder = "/data/home/sjkang/publish"

# 각 영상의 절대 경로
video1 = "/data/home/khjw1835/sjkang_test/videos/kevin_perso3-Keinv_GraySuit_Looping_Green.mp4.crop_70.webm_kevin_perso3_step_000000624000_231124_devin_front_config-nayoonk_front_v1_glasses_fo.webm"
video2 = "/data/home/khjw1835/sjkang_test/videos/kevin_perso3-Kevin_GraySuit_3mins_Greeen.mp4.crop_70.webm_kevin_perso3_step_000000624000_231124_devin_front_config-nayoonk_front_v1_glasses_fo.webm"

# fps 설정
fps = 24

In [2]:
# 버전 1.0.0 (첫 release)
from IPython.display import display, HTML, Javascript
import os

# 파일명을 취득
filename1 = os.path.basename(video1)
filename2 = os.path.basename(video2)

# ipynb(작업 디렉토리) 기준 상대경로 계산
video1 = os.path.relpath(video1, present_folder)
video2 = os.path.relpath(video2, present_folder)

# HTML 코드
display(HTML(f"""
<head>
    <!-- 버튼 아이콘 링크 -->
    <link 
        rel="stylesheet" 
        href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.7.2/css/all.min.css" 
        integrity="sha512-Evv84Mr4kqVGRNSgIGL/F/aIDqQb7xQ2vcrdIwxfjThSH8CSR7PBEakCr51Ck+w+/U6swU2Im1vVX0SVk9ABhg==" 
        crossorigin="anonymous" 
        referrerpolicy="no-referrer"
    />
</head>

<style>
    .video-container {{
        position: relative;
        display: inline-block;
        transition: border 0.3s ease;
    }}
    .zoom-rect {{
        position: absolute;
        border: 2px dashed red;
        pointer-events: none;
        z-index: 10;
    }}
    video {{
        position: absolute;
        top: 0;
        left: 0;
        transform-origin: top left;
    }}
    .control-center {{
        text-align: center;
        margin-top: 30px;
    }}
    .control-center button {{
        height: 40px;
        padding: 10px 20px;
        font-size: 16px;
        margin: 5px;
        cursor: pointer;
    }}
    .filename-container {{
        width: 720px; 
        padding: 0px 5px 0px 5px;
        word-break: break-word; 
        overflow-wrap: break-word; 
        font-size: 14px; color: #444; 
        text-align: left;
    }}
</style>

<div style="display: flex; justify-content: center; margin: 0; padding: 0;">
    <!-- 왼쪽 영상 -->
    <div style="text-align: center;">
        <!-- 파일명과 프레임 박스 -->
        <div style="margin-top: 10px;">
            <div class="filename-container">
                파일명: {filename1}
            </div><br>
            <label>시작 프레임: <input type="number" id="start1" value="0" style="width:60px;"></label>
            <div id="frameDisplay1" style="font-size: 14px; color: #666;">현재 프레임: 0</div>
        </div>
        <!-- 영상 박스 -->
        <div class="video-container" id="container1" style="width:720px; height:540px; overflow:hidden;" ondragover="event.preventDefault()">
            <video id="video1" width="720" height="540" controls>
                <source src="{video1}" type="video/webm">
                Your browser does not support the video tag.
            </video>
            <div class="zoom-rect" id="rect1" style="display:none;"></div>
        </div>
    </div>
    
    <!-- 오른쪽 영상 -->
    <div style="text-align: center;">
        <!-- 파일명과 프레임 박스 -->
        <div style="margin-top: 10px;">
            <div class="filename-container">
                파일명: {filename2}
            </div><br>
            <label>시작 프레임: <input type="number" id="start2" value="0" style="width:60px;"></label>
            <div id="frameDisplay2" style="font-size: 14px; color: #666;">현재 프레임: 0</div>
        </div>
        <!-- 영상 박스 -->
        <div class="video-container" id="container2" style="width:720px; height:540px; overflow:hidden;" ondragover="event.preventDefault()">
            <video id="video2" width="720" height="540" controls>
                <source src="{video2}" type="video/webm">
                Your browser does not support the video tag.
            </video>
            <div class="zoom-rect" id="rect2" style="display:none;"></div>
        </div>
    </div>
</div>

<!-- 중앙 제어 버튼 -->
<div class="control-center">
    <!-- 동시 컨트롤 재생바 -->
    <div style="display: flex; align-items: center; justify-content: center; gap: 10px;">
        <div id="currentTime" style="font-size: 14px;">00:00</div>
        <input type="range" id="syncSlider" min="0" value="0" max="100" step="1" style="width: 720px;">
        <div id="duration" style="font-size: 14px;">00:00</div>
    </div>

    <!-- ⏪ 1프레임 뒤 이동 버튼 -->
    <button id="rewindBtn" onclick="
        const v1 = document.getElementById('video1');
        const v2 = document.getElementById('video2');
        v1.currentTime = Math.max(0, v1.currentTime - (1/{fps}));
        v2.currentTime = Math.max(0, v2.currentTime - (1/{fps}));
    "><i class="fa-solid fa-backward"></i></button>

    <!-- ⏯️ 재생/일시정지 버튼 -->
    <button id="playPauseBtn" onclick="
        const v1 = document.getElementById('video1');
        const v2 = document.getElementById('video2');
        if (v1.paused && v2.paused) {{
            v1.play();
            v2.play();
        }} else {{
            v1.pause();
            v2.pause();
        }}
    "><i class="fa-solid fa-play"></i><i class="fa-solid fa-pause"></i></button>

    <!-- ⏩ 1프레임 앞 이동 버튼 -->
    <button id="forwardBtn" onclick="
        const v1 = document.getElementById('video1');
        const v2 = document.getElementById('video2');
        v1.currentTime = Math.min(v1.duration, v1.currentTime + (1/{fps}));
        v2.currentTime = Math.min(v2.duration, v2.currentTime + (1/{fps}));
    "><i class="fa-solid fa-forward"></i></button>
</div>
"""))

# 자바 스크립트 코드
Javascript(f"""
// 동시 확대 기능 함수
function enableSynchronizedZoom(video1Id, container1Id, rect1Id, video2Id, container2Id, rect2Id) {{
    const container1 = document.getElementById(container1Id);
    const video1 = document.getElementById(video1Id);
    const rect1 = document.getElementById(rect1Id);
    
    const container2 = document.getElementById(container2Id);
    const video2 = document.getElementById(video2Id);
    const rect2 = document.getElementById(rect2Id);
    
    let startX = 0, startY = 0;
    let zoomed = false;
    let isDragging = false;
    let currentLeft = 0, currentTop = 0;
    // 클릭과 드래그 구분용(클릭으로 간주할 수 있는 최대 이동 거리)
    let clickThreshold = 5;
    let mouseDownX = 0, mouseDownY = 0;
    
    // mousedown시 동작(축소/드래그 이동)
    container1.onmousedown = function(e) {{

        startX = e.offsetX;
        startY = e.offsetY;

        // 확대된 상황일 때(드래그 or 축소)
        if (zoomed) {{
            // 마우스 버튼을 처음 누른 위치 
            mouseDownX = e.clientX;
            mouseDownY = e.clientY;
            // 현재 드래그 중인지 여부 플래그
            isDragging = false;
            currentLeft = parseInt(video1.style.left) || 0;
            currentTop = parseInt(video1.style.top) || 0;

            // 드래그 이동 중(마우스가 움직일 때 마다 이 함수가 호출됨)
            document.onmousemove = function(e2) {{
                const dx = e2.clientX - mouseDownX;
                const dy = e2.clientY - mouseDownY;
        
                if (!isDragging && (Math.abs(dx) > clickThreshold || Math.abs(dy) > clickThreshold)) {{
                    isDragging = true;
                }}

                // 마우스가 움직일 때마다 위치 갱신
                if (isDragging) {{
                    [video1, video2].forEach(video => {{
                        video.style.left = `${{currentLeft + dx}}px`;
                        video.style.top = `${{currentTop + dy}}px`;
                    }});
                }}
            }};
            
            // 드래그 종료
            document.onmouseup = function(e2) {{
                const moveX = Math.abs(e2.clientX - mouseDownX);
                const moveY = Math.abs(e2.clientY - mouseDownY);
    
                if (moveX < clickThreshold && moveY < clickThreshold) {{
                    // 거의 움직이지 않았다면 → 클릭으로 간주하고 축소 실행
                    [video1, video2].forEach(video => {{
                        video.style.transform = "";
                        video.style.left = "0";
                        video.style.top = "0";
                    }});
                    zoomed = false;
                }}
            
                isDragging = false;
                document.onmousemove = null;
                document.onmouseup = null;
            }};
        // 확대되지 않은 상황일 때
        }} else {{ 
            [rect1, rect2].forEach(rect => {{
                rect.style.left = startX + "px";
                rect.style.top = startY + "px";
                rect.style.width = "0px";
                rect.style.height = "0px";
                rect.style.display = "block";
            }});
            // 확대 시작
            container1.onmousemove = function(e2) {{
                // 확대 영역 선택 중
                const width = e2.offsetX - startX;
                const height = e2.offsetY - startY;
                const left = (width < 0 ? e2.offsetX : startX);
                const top = (height < 0 ? e2.offsetY : startY);
                const w = Math.abs(width) + "px";
                const h = Math.abs(height) + "px";
                
                [rect1, rect2].forEach(rect => {{
                    rect.style.left = left + "px";
                    rect.style.top = top + "px";
                    rect.style.width = w;
                    rect.style.height = h;
                }});
            }};

            container1.onmouseup = function(e3) {{                
                container1.onmousemove = null;
                container1.onmouseup = null;
                [rect1, rect2].forEach(rect => rect.style.display = "none");

                // 위치 정보 취득
                const endX = e3.offsetX;
                const endY = e3.offsetY;

                // 선택 영역의 왼쪽 상단 좌표
                const left = Math.min(startX, endX);
                const top = Math.min(startY, endY);

                // 선택 영역의 크기
                const zoomWidth = Math.abs(endX - startX);
                const zoomHeight = Math.abs(endY - startY);

                // 확대 비율 계산
                const scaleX = container1.offsetWidth / zoomWidth;
                const scaleY = container1.offsetHeight / zoomHeight;
                const scale = Math.min(scaleX, scaleY);

                // 비디오 확대 적용
                [video1, video2].forEach(video => {{
                    video.style.transform = `scale(${{scale}})`;
                    video.style.left = `-${{left * scale}}px`;
                    video.style.top = `-${{top * scale}}px`;
                }});
                
                zoomed = true;
            }};
        }}
    }};

    // 마우스가 컨테이너를 벗어났을 때 드래그 중지
    container1.onmouseleave = function() {{
        isDragging = false;
    }};
}}

// 비디오 기본 클릭 동작 비활성화 함수
function disableVideoClick(videoId) {{
    const video = document.getElementById(videoId);
    video.addEventListener('click', function(e) {{
        // 기본 동작 방지: 비디오의 기본 클릭 동작(재생/일시정지 토글)을 막음
        e.preventDefault();
        // 클릭 이벤트가 상위 요소로 전파되는 것을 방지, 다른 이벤트 핸들러가 실행되는 것을 막음
        e.stopPropagation();
    }});
}}

// 각 영상 드래그 앤 드롭 함수
function setupDragAndDrop() {{
    // 왼쪽 컨테이너
    const container1 = document.getElementById('container1');
    const video1 = document.getElementById('video1');
    
    // 오른쪽 컨테이너
    const container2 = document.getElementById('container2');
    const video2 = document.getElementById('video2');

    // 왼쪽 컨테이너 이벤트
    container1.addEventListener('dragover', function(e) {{
        e.preventDefault();
        e.stopPropagation();
        // 테두리 추가
        container1.style.border = '2px dashed #666';
    }});
    
    container1.addEventListener('dragleave', function(e) {{
        e.preventDefault();
        e.stopPropagation();
        // 테두리 제거
        container1.style.border = 'none';
    }});

    // 파일을 드롭 할 때 실행되는 drop listener 함수
    container1.addEventListener('drop', function(e) {{
        // 기본 동작과 이벤트 전파를 막음(브라우저의 기본 파일 드롭 동작 막음)
        e.preventDefault();
        e.stopPropagation();
        // 테두리 제거
        container1.style.border = 'none';

        // 드롭한 파일을 가져와 영상인지 확인 후 URL 부여
        const files = e.dataTransfer.files;
        if (files.length > 0) {{
            const file = files[0];
            if (file.type.startsWith('video/')) {{
                const source = video1.querySelector('source');
                source.src = URL.createObjectURL(file);
                video1.load();
            }}
        }}
    }});
    
    // 오른쪽 컨테이너 이벤트
    container2.addEventListener('dragover', function(e) {{
        e.preventDefault();
        e.stopPropagation();
        // 테두리 추가
        container2.style.border = '2px dashed #666';
    }});
    
    container2.addEventListener('dragleave', function(e) {{
        e.preventDefault();
        e.stopPropagation();
        // 테두리 제거
        container2.style.border = 'none';
    }});

    // 파일을 드롭 할 때 실행되는 drop listener 함수
    container2.addEventListener('drop', function(e) {{
        // 기본 동작과 이벤트 전파를 막음(브라우저의 기본 파일 드롭 동작 막음)
        e.preventDefault();
        e.stopPropagation();
        // 테두리 제거
        container2.style.border = 'none';

        // 드롭한 파일을 가져와 영상인지 확인 후 URL 부여
        const files = e.dataTransfer.files;
        if (files.length > 0) {{
            const file = files[0];
            if (file.type.startsWith('video/')) {{
                const source = video2.querySelector('source');
                source.src = URL.createObjectURL(file);
                video2.load();
            }}
        }}
    }});
}}    

// 함수 실행
enableSynchronizedZoom('video1', 'container1', 'rect1', 'video2', 'container2', 'rect2');
disableVideoClick('video1');
disableVideoClick('video2');
// 드래그 앤 드롭 기능 초기화
setupDragAndDrop();

// video1의 현재 프레임을 업데이트
document.getElementById('video1').addEventListener('timeupdate', function() {{
    document.getElementById('frameDisplay1').innerText = `현재 프레임: ` + Math.floor(this.currentTime * {fps});
}});

// video2의 현재 프레임을 업데이트
document.getElementById('video2').addEventListener('timeupdate', function() {{
    document.getElementById('frameDisplay2').innerText = `현재 프레임: ` + Math.floor(this.currentTime * {fps});
}});

// 슬라이더 관련 코드 (즉시 실행 함수)
(function() {{
    // 슬라이더 엘리먼트 가져오기
    const syncSlider = document.getElementById('syncSlider');
    const video1 = document.getElementById('video1');
    const video2 = document.getElementById('video2');

    // 시작 프레임 값을 가져와 영상 시작점 설정
    function getStartFrames() {{
        return [
            parseInt(document.getElementById('start1').value) || 0,
            parseInt(document.getElementById('start2').value) || 0
        ];
    }}

    // 프레임 카운트 함수
    function getFrameCount(duration, startFrame) {{
        return Math.floor(duration * {fps}) - startFrame;
    }}

    function getSyncInfo() {{
        const [start1, start2] = getStartFrames();
        const duration1 = video1.duration || 0; // 비디오가 로드되지 않았을 경우 0 사용
        const duration2 = video2.duration || 0;

        // 프레임 수 계산
        const frameCount1 = getFrameCount(duration1, start1);
        const frameCount2 = getFrameCount(duration2, start2);

        // 어떤 비디오를 기준으로 동기화할 지 결정
        const isVideo1Base = start1 >= start2;
        const baseStart = isVideo1Base ? start1 : start2;
        
        const totalFrames = Math.min(frameCount1, frameCount2);

        return {{ isVideo1Base, baseStart, totalFrames, start1, start2 }};
    }}

    // 시간 포맷팅 함수
    function formatTime(seconds) {{
        const minutes = Math.floor(seconds / 60);
        const remainingSeconds = Math.floor(seconds % 60);
        return `${{minutes.toString().padStart(2, '0')}}:${{remainingSeconds.toString().padStart(2, '0')}}`;
    }}       
    
    // 슬라이더 조작 → 두 영상 시간 변경
    syncSlider.addEventListener('input', function() {{
        const {{ totalFrames, start1, start2 }} = getSyncInfo();
        const currentFrame = (syncSlider.value / 100) * totalFrames;

        video1.currentTime = (start1 + currentFrame) / {fps};
        video2.currentTime = (start2 + currentFrame) / {fps};

        video1.currentTime = currentTime1;
        video2.currentTime = currentTime2;
        
        // 현재 시간 업데이트
        document.getElementById('currentTime').textContent = formatTime(currentTime1);
    }});

    // 비디오 로드 시 전체 시간 표시
    video1.addEventListener('loadedmetadata', function() {{
        document.getElementById('duration').textContent = formatTime(video1.duration);
    }});

    // 비디오 재생 중 현재 시간 업데이트
    video1.addEventListener('timeupdate', function() {{
        const {{ isVideo1Base, start1 }} = getSyncInfo();
        if (isVideo1Base) {{
            updateSliderFromBaseVideo(video1, start1);
            document.getElementById('currentTime').textContent = formatTime(video1.currentTime);
        }}
    }});

    video2.addEventListener('timeupdate', function() {{
        const {{ isVideo1Base, start2 }} = getSyncInfo();
        if (!isVideo1Base) {{
            updateSliderFromBaseVideo(video2, start2);
            document.getElementById('currentTime').textContent = formatTime(video2.currentTime);
        }}
    }});

    // 슬라이더는 기준 영상 기준으로만 갱신
    function updateSliderFromBaseVideo(video, startBase) {{
        const {{ baseStart, totalFrames }} = getSyncInfo();
        const currentFrame = (video.currentTime * {fps}) - baseStart;
        const value = Math.max(0, Math.min(100, (currentFrame / totalFrames) * 100));
        syncSlider.value = value;
    }}

    video1.addEventListener('timeupdate', function() {{
        const {{ isVideo1Base, start1 }} = getSyncInfo();
        if (isVideo1Base) {{
            updateSliderFromBaseVideo(video1, start1);
        }}
    }});

    video2.addEventListener('timeupdate', function() {{
        const {{ isVideo1Base, start2 }} = getSyncInfo();
        if (!isVideo1Base) {{
            updateSliderFromBaseVideo(video2, start2);
        }}
    }});
}})();      

// 키보드 단축키 설정 코드 (즉시 실행 함수)
(function() {{
    // 중복 등록 방지
    if (window._videoKeyHandler) {{
        window.removeEventListener('keydown', window._videoKeyHandler, true);
    }}

    // 키보드 단축키 설정 이벤트 핸들러 함수
    window._videoKeyHandler = function(e) {{
        // 포커스가 인풋 필드나 텍스트에 있을 경우는 제외(사용자 입력에 방해되지 않도록 하는 코드)
        const tag = document.activeElement.tagName.toLowerCase();
        if (tag === 'input' || tag === 'textarea') return;

        // ID로 비디오 속성 취득
        const v1 = document.getElementById('video1');
        const v2 = document.getElementById('video2');

        // 입력창 포커스 시 무시
        if (e.target.tagName === 'INPUT' || e.target.tagName === 'TEXTAREA' || e.target.isContentEditable) {{
            return;
        }}

        if (e.code === 'Space') {{
            // 스페이스바 기본 동작(페이지 아래로 스크롤) 차단
            e.preventDefault();

            // 포커스된 요소가 <video>일 경우, 동영상 기본 동작(재생/일시정지)을 막기 위해 강제로 blur 처리
            if (document.activeElement.tagName.toLowerCase() === 'video') {{
                document.activeElement.blur();
            }}

            // 두 비디오의 상태를 확인
            const bothPaused = v1.paused && v2.paused;
            
            // 두 비디오를 동시에 제어
            if (bothPaused) {{
                v1.play();
                v2.play();
            }} else {{
                v1.pause();
                v2.pause();
            }}
        }} else if (e.code === 'ArrowLeft' || e.code === 'ArrowRight') {{  // 방향키로 뒤로 감기/앞으로 감기 기능하도록 설정.
            // 방향키 누르면 페이지가 움직이는 기본 동작 차단
            e.preventDefault();
            // 다른 곳에 이벤트가 전달되지 않게 함.
            e.stopPropagation();
            
            const btn = e.code === 'ArrowLeft' ? 
                document.getElementById("rewindBtn") : 
                document.getElementById("forwardBtn");
                
            if (btn) {{
                btn.click();
            }}
        }}
    }};     

    // 새 이벤트 핸들러 등록
    window.addEventListener('keydown', window._videoKeyHandler, true);
}})();
""")

<IPython.core.display.Javascript object>